# Het internet als databron # 

## Introductie webscraping met Jupyter en Python ## 
[**Ed. de Feber**](mailto:e.defeber@horizoncollege.nl)  
ROC Horizon College

---

Ik wil jullie in deze korte introductie WebScraping laten zien hoe je met Jupyter en Python vrij eenvoudig data kan halen uit bestaande bronnen op het internet.

Ik heb de introductie als volgt ingedeeld



### 1. Python en Jupyter? ### 
### 2. WebScraping? It's all about the data ###
### 3. Verschillende manieren om data binnen te halen en te gebruiken ###
- Bestaande API's gebruiken die vaak door de social networks worden geleverd (Facebook, Twitter, Linked In, ...). Valt buiten deze introductie.
- Bestaande Tabellen gebruiken
- Content uit nieuwspagina's, Wikipedia, enz binnen halen 
- Inlezen van openbare bronbestanden, meestal in excel, csv formaat, die aan ons ter beschikking worden gesteld door een derde partij. Keuze uit DUO, CBS, MBO-Raad, SBB, )
- Het ontleden van een webpagina of volledige websites, om daar relevante data uit te halen (Stages, Beroepsprofielen, Vacatures, ...).

## 1. Python en Jupyter ##

     

#### **Python**
Een programeertaal dat over het algemeen als de standaard wordt gezien gezien in de wereld van DataScience en AI. Alle grote spelers zoals Google, Spotify, Amazon, Facebook, enz. maken gebruik van Python. Mooi om te vermelden is dat Python in de jaren 90 van de vorige eeuw is bedacht en ontwikkeld door een Nederlander **Guido van Rossem**
( ik meen zelfs dat hij op dat moment nog student aan de UvA was ). 

Vorig jaar is Guido ingelijfd door Microsoft dus het moge duidelijk zijn dat ook Microsoft Python omarmd heeft. 

#### **Jupyter**
Is het platform waar jullie nu tegenaan kijken. Het is een zogenaamde NOTEBOOK, en wordt zeer vaak gebruikt door Dataspecialisten om inzichten, resultaten aan opdrachtgevers te presenteren. Een Jupyter notebook kan gedeeld worden op het internet en biedt de mogelijkheid om op 1 plaats data afkomstig van meerdere verschillende bronnen te presenteren. 

**Deze Jupyter notebook wordt met jullie gedeeld op:**

[MijnNotebooek](https://www.example.com)

---



## 2. WebScraping ##

In deze introductie interpreteer ik het begrip Webscraping heel ruim als een verzameling van methoden en technieken om data uit internetbronnen te halen. Van het inlezen van een HTML Tabel die al kant en klaar op een webpagina te vinden is, via het gebruiken van openbare databronnen meestal in Excel of CSV formaat, tot aan het uit een rafelen van een complete website en/of het combineren van dit alles en hierbij data uit meerdere internetsites te combineren. Ik laat dit voor het gemak hier samenvallen onder het begrip WebScraping. 

En waarom niet? 
**It's all about the resulting data!**




#### Aan de slag ### 

1.  **Inlezen en manipuleren van data**
2.  **Presenteren en visualiseren van resultaten** 

Bijna altijd als je iets met Python wilt doen maak je gebruik van een of meer bibliotheken. Dat is tevens de grote kracht van **Python**, omdat **voor** praktische **elke situatie** wel **een bibliotheek** te vinden is. Hieronder **importeren** we een aantal bibliotheken* waaronder pandas en matplotlib. Panda is de held als het gaat om statistieken en Matrix rekenen en Matplotlib is de absolute grootheid als het gaat om Grafieken e.d.  


In [ ]:
!pip install pandas numpy requests_html openpyxl matplotlib bqplot newspaper3k
import pandas as pd
import requests
from requests_html import HTMLSession
from ipywidgets import Image
# import pyxll
import openpyxl

%matplotlib inline
import ipywidgets as widgets
import bqplot.pyplot as plt

## 3. Verschillende manieren om data binnen te halen en te gebruiken ##

### Laten we beginnen met een artikel uit een krant te halen ###

**NU.NL**

Hiervoor importeer ik bibliotheek newspaper in Python. Deze bibliotheek kan voor praktisch elke nieuwssite gebruikt worden en kent semantische entiteiten als Artikel, Auteur, enz.

### Artikel van nu.nl met "Newspaper bibliotheek" ###


In [ ]:
from newspaper import Article
from requests_html import HTMLSession

De eerste stap is om tegen Python vertellen welke url ik wil binnen halem. Vervolgs maakt Python daar dan een connectie mee.

**Hieronder een artikel dat ik afgelopen zondag vond**

![alt text](./images/nunl3.png "nu.nl afgelopen zondag")


**Deze pagina heeft de volgende url**

In [ ]:
url = "https://www.nu.nl/coronavirus/6177311/voor-het-eerst-sinds-half-november-minder-dan-vierhonderd-coronapatienten-op-ic.html"

Met de newspaper bibliotheek hebben we de beschikking over een zogenaamd **Article Object**.

In drie stappen kunnen we hiermee een krantenartikel binnen halen:
1. Eerst maken we een Article object, die we article noemen.
2. Dan gaan we de pagina downloaden,
3. en tenslotte parsen.

In [ ]:
article = Article(url)
article.download()
article.parse()

**Het article object bewaart alle resultaten.**

De belangrijkste zijn **title, authors, publish_date en text**. 
Ik zal ze hieronder tonen.

In [ ]:
article.title

In [ ]:
article.authors

Authors returns a list. In this case, it only has one item. 

In [ ]:
article.publish_date

In this case, publish_date returns the publication date and time converted into a Python datetime object. 

In [ ]:
print(article.text)

**Zoals jullie zien hebben we de volledige tekst van het artikel teruggekregen.**

In [ ]:
inhoud = article.text

woordenlijst = []

for woord in inhoud.split():
    if woord.lower() not in "nog de dan het in een en er te ten op van het voor is met dat dag":
        woordenlijst.append((woord.lower(),inhoud.count(woord)))
    
woordenlijst = sorted(list(set(woordenlijst)))
    
woordenlijst = pd.DataFrame(woordenlijst,columns=["WOORD","FREQUENTIE"])
woordenlijst = woordenlijst.sort_values(by="FREQUENTIE", ascending=False)

woordenlijst = woordenlijst[["WOORD", "FREQUENTIE"]][:20]
woordenlijst

In [ ]:
woordenlijst = woordenlijst.groupby(by="FREQUENTIE")
woordenlijst.all()

Als je **meerdere pagina's** wilt opslaan dan is het makkelijker om dit proces te automatiseren. Daarom maak ik hieronder een functie die als input een URL nodig heeft en vervolgens de meta-data en html code teruggeeft. 

In [ ]:
def get_article_info(url):
    """Download and parse a newspaper url."""
    article = Article(url)
    article.download()
    article.parse()

    article_details = {
        "title": article.title,
        "text": article.text,
        "webUrl": article.url,
        "authors": article.authors,
        "html": article.html,
        "date": article.publish_date,
        "description": article.meta_description,
    }
    return article_details

Even Checken of het werkt!

In [ ]:
url = 'https://www.nu.nl/schaatsen/6177313/nuis-verslaat-krol-op-1500-meter-en-is-voor-het-eerst-europees-kampioen.html'
a = get_article_info(url)
print(a["title"])
print(a["authors"])
print(a["text"])


We kunnen dit artikel nu ook in een panda gooien

In [ ]:
df = pd.DataFrame.from_records(a)

In [ ]:
df.head()
# df['text'].text()

In [ ]:
df.to_json("ap_articles.json", orient="records")

While this might seem like overkill for one article, it scales up quite nicely if you have a longer list of URLs.

In [ ]:
urls = [
    "https://www.nu.nl/tech/6177312/ces-trekt-veel-minder-bezoekers-dan-voor-coronapandemie.html",
    "https://www.nu.nl/schaatsen/6177313/nuis-verslaat-krol-op-1500-meter-en-is-voor-het-eerst-europees-kampioen.html",
    "https://www.nu.nl/coronavirus/6177311/voor-het-eerst-sinds-half-november-minder-dan-vierhonderd-coronapatienten-op-ic.html",
]

In [ ]:
article_data = []  # Blank list to store results

# Loop over each URL
for url in urls:
    a = get_article_info(url)
    article_data.append(a)

# convert list of dictionaries to dataframe
df = pd.DataFrame.from_records(article_data)
df.columns

df_beter = df[['title', 'text', 'webUrl', 'description']].copy()

df_beter

In [ ]:
df[["title", "webUrl"]]

### Wat als we iets anders willen dan de inhoud van een artikel? ###



In [ ]:
    

# alle_links[12].html

In [ ]:
url = 'https://nu.nl/'

In [ ]:
session = HTMLSession()

r = session.get(url)

In [ ]:
parsed_html = r.html

In [ ]:
parsed_html

In [ ]:
parsed_html.html

### HTML code doorzoeken  ###

We zijn op zoek naar de headlines en er bestaat een Tag **headline** dus we gaan alle tags met de naam headlines binnen halen.
    



We gebruiken hier de functie find voor

In [ ]:
for item in parsed_html.find('.headline'):
    print(item.text)


### Hoe krijg je nu de links van de headlines? ###

Daarvoor gebruiken we de functie **absolute_links**

In [ ]:
mijnlinks = []
for link in parsed_html.absolute_links:
    if 'corona' in link: # and 'covid' in link:
        mijnlinks.append(link)

df = pd.DataFrame(mijnlinks,columns =["Mijnlinks"])



#### In df staat nu de Matrix met de links in soort tabelvorm ####

Daarvoor gebruiken we de functie **absolute_links**

In [ ]:
df


#### Met de functie **absolute_links** maken we echet hyperlinks ####



In [ ]:
for link in parsed_html.absolute_links:
    if 'corona' in link: # and 'covid' in link:
        
        print(link)
        print()

In [ ]:
widgets.HTML("<a href='" + df_beter["webUrl"][0] + "'>'" + df_beter["webUrl"][0] + "</a>")

---




In [ ]:
session = HTMLSession()
url = "https://www.nu.nl/net-binnen"
r = session.get(url)
r.status_code

Als de status 200 is betekent dat dat het binnenhalen gelukt is.
Ik zal de inhoud tonen...

In [ ]:
parsed_html = r.html

In [ ]:
parsed_html

In [ ]:
parsed_html.html

Uit deze brij moeten we de HTML tags zoeken die we nodig hebben.

In [ ]:
parsed_html.find("div, class")

In [ ]:
ruwe_datakoppen = parsed_html.find('div, class')

In [ ]:
krantenkoppen = []

for kop in ruwe_datakoppen:
    kop = kop.find('.title')
    if len(kop) > 0:
        for kopje in kop:
            krantenkoppen.append(kopje.text)
    else:
        continue
        

In [ ]:
koppen = ""

if len(krantenkoppen) > 0:
    krantenkoppen = set(krantenkoppen)
    for kop in krantenkoppen:
        koppen += "<li>" + kop + "</li>"    




widgets.HTML(
    
    value = koppen,
    # placeholder='Some HTML',
    # description='Some HTML',
)


In [ ]:
    

# alle_links[12].html

# Bestaande tabellen gebruiken #

Als voorbeeld nemen we de site van de **MBORAAD**


Image.from_file("images/mboraad.png")

In [ ]:

Image.from_file("images/mboraad.png")

## https://www.mboraad.nl/het-mbo/feiten-en-cijfers/studenten-het-mbo ##

In [ ]:

mboraad_url = 'https://www.mboraad.nl/het-mbo/feiten-en-cijfers/studenten-het-mbo'

mboraad2_url = 'https://www.mboraad.nl/het-mbo/feiten-en-cijfers/aansluiting-op-de-arbeidsmarkt'
cbs_url = 'https://opendata.cbs.nl/statline/#/CBS/nl/dataset/83851NED/table?ts=1641657271358'

pd.read_html(mboraad_url)

Bronnen van internet

In [ ]:
html_table_list = pd.read_html(mboraad_url)
html_table_list

In [ ]:
tabel1 = html_table_list[0]
# tabel2 = html_table_list[1]
type(tabel1)
tabel1.values

In [ ]:
# tabel2.values

## https://www.mboraad.nl/het-mbo/feiten-en-cijfers/aansluiting-op-de-arbeidsmarkt ## 

### *TABEL: Aansluiting van opleidingen met huidige functie is voldoende/goed (%)* ###
---

In [ ]:
html_table_list = pd.read_html(mboraad2_url)
html_table_list[0]

### *TABEL Werkloosheid %* ###
---



In [ ]:
html_table_list[1]

In [ ]:
tabel1 = html_table_list[0]
tabel2 = html_table_list[1]
type(tabel1)
tabel1.values

In [ ]:
tabel2.values

In [ ]:
grafiekje = pd.DataFrame(tabel1.values, tabel2.values)

grafiekje[:2].plot()

## BESTANDEN VAN DE DUO  ##

In [ ]:
df = pd.read_csv('https://duo.nl/open_onderwijsdata/images/01-studenten-per-instelling-bestuur-plaats-gemeente-provincie-type-mbo-2016-2020.csv',sep=';',encoding="latin")
df.head()


Bronnen van internet

In [ ]:
x = list(df["INSTELLINGSNAAM"])[:5]
x

In [ ]:
df_instelling = df[df["INSTELLINGSNAAM"]=="ROC Drenthe College"].copy()
df_instelling.head()

In [ ]:
df_instelling[["PEILJAAR", "BBL    "]].plot.bar(x="PEILJAAR",y="BBL    ")
# df_instelling.columns

In [ ]:
df_grootste = df[df["TOTAAL"] == df["TOTAAL"].max()].copy()
df_grootste.head()

In [ ]:
import numpy as np
import bqplot.pyplot as plt

dates = df_instelling["PEILJAAR"]



In [ ]:

size = len(dates)

totaal = df_instelling["TOTAAL"] 

titel = df_instelling["INSTELLINGSNAAM"][0] + ' Aantal Studenten'

In [ ]:
fig = plt.figure(title=titel, background_style={'fill': 'lightblue'},
                 title_style={'font-size': '20px','fill': 'DarkOrange'})
axes_options = {'x': {'label': 'Cohort', 'tick_format': 'i'},
                'y': {'label': 'Aantal Studenten', 'tick_format': '0.0f'}}
plt.plot(dates, totaal, 'b', axes_options=axes_options) # third argument is the marker string
fig

In [ ]:
y = df["TOTAAL"] 
x = list(df["INSTELLINGSNAAM"][:5])

# 1. Create the figure object
fig = plt.figure(title='Aantal Studenten per instelling')

# 2. Customize the axes options
axes_opts = {'x': {'label': 'INSTELLING', 'grid_lines': 'none'},
             'y': {'label': 'AANTAL STUDENTEN', 'tick_format': 't'}}

# 3. Create a Bars mark by calling plt.bar function
bar = plt.bar(x=x, y=y, padding=.1, axes_options=axes_opts)

# 4. directly display the figure object created in step 1 (note that the toolbar no longer shows up)
fig

In [ ]:
y = df["BBL    "] / df["TOTAAL"] *100

# y = dfdf["BBL    "]]

x = list(df["INSTELLINGSNAAM"][35:43])

# 1. Create the figure object
fig = plt.figure(title='Percentage BBL studenten')

# 2. Customize the axes options
axes_opts = {'x': {'label': 'INSTELLING', 'grid_lines': 'none'},
             'y': {'label': 'PERCENTAGE BBL STUDENTEN', 'tick_format': 't' }}

# 3. Create a Bars mark by calling plt.bar function
bar = plt.bar(x=x, y=y, padding=.1, axes_options=axes_opts)

# 4. directly display the figure object created in step 1 (note that the toolbar no longer shows up)
fig

In [ ]:
y_data_2 =  df["BOLVT  "]
y_data_3 = df["BBL    "]
y_data = df["TOTAAL"]

plt.figure(title='STUDENTEN LEERWEG')


plt.scatter(y_data_2, y_data_3, color=y_data, stroke='black')
plt.show()



In [ ]:
import numpy as np

df_instelling = df[df["INSTELLINGSNAAM"]=="ROC Drenthe College"].copy()
df_instelling.head()
x_data = df_instelling["INSTELLINGSNAAM"]
# np.cumsum(np.random.randn(size) * 100.0)
y_data = np.absolute(df["BBL    "]/ df["TOTAAL"]) * 100

plt.figure()
plt.hist(y_data, colors=['OrangeRed'])
plt.show()

In [ ]:
plt.figure()
d = abs(df["TOTAAL"][:5])
plt.pie(d, labels = ["2016","2017","2018","2019","2020"], )
plt.show()


In [ ]:
plt.figure()
plt.geo(map_data='WorldMap')
plt.show()

In [ ]:
df = pd.read_csv('https://duo.nl/open_onderwijsdata/images/02-combinatie-crebo-en-beroep-2020-2021.csv',sep=';',encoding="latin")
df.head()


In [ ]:
df.sample(7)

In [ ]:
df = pd.read_excel(r'https://duo.nl/open_onderwijsdata/images/03-erkende-niet-bekostigde-opleidingen.xlsx')
df.head()


# https://duo.nl/open_onderwijsdata/images/2021-kwaliteitsafspraken-per-instelling.xlsx

In [ ]:
df_albeda = df[df["SCHOOLNAAM"] == "Stichting Albeda"].copy()
df_albeda[["SCHOOLNAAM","NAAM OPLEIDING", "CREBONUMMER"]]

In [ ]:
opleidingen = df_albeda["NAAM OPLEIDING"]
len(opleidingen)